In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-18 15:39 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 15:49 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 16:10 /tmp/output
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 15:39 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 15:49 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 16:10 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 16:26 /tmp/wordcount


In [4]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

In [6]:
%%hive

DROP TABLE IF EXISTS data;
DROP TABLE IF EXISTS sorted_data;

DROP TABLE IF EXISTS data;
OK
Time taken: 5.471 seconds
DROP TABLE IF EXISTS sorted_data;
OK
Time taken: 0.045 seconds


In [7]:
%%hive

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 1.98 seconds
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;
Loading data to table default.data
OK
Time taken: 3.108 seconds


In [11]:
%%hive
CREATE TABLE sorted_data AS
(SELECT * FROM data ORDER BY letter, numb);

CREATE TABLE sorted_data AS
(SELECT * FROM data ORDER BY letter, numb);
Query ID = root_20200218163452_409a2a2a-8d1f-4a67-94a3-4a3c83a84741
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582040149898_0006, Tracking URL = http://900d27d24b85:8088/proxy/application_1582040149898_0006/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582040149898_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 16:35:11,506 Stage-1 map = 0%,  reduce = 0%
2020-02-18 16:35:24,013 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.76 sec
2020-02-18 16:35:37,763 Stage-1 map = 100%,  reduce = 100%, Cumulativ

In [12]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM sorted_data;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM sorted_data;
Query ID = root_20200218163547_4449f7d1-b431-46b8-8673-e6ea12604f64
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582040149898_0007, Tracking URL = http://900d27d24b85:8088/proxy/application_1582040149898_0007/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582040149898_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 16:36:06,533 Stage-1 map = 0%,  reduce = 0%
2020-02-18 16:36:18,837 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.87 sec
MapReduce Total cumulative CPU time: 3 seconds 870 msec
Ended Job = job_1582040149898_0007
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_

In [13]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [14]:
!hdfs dfs -cat /tmp/output/000000_0 | head

A,1990-10-06,10
A,1990-09-05,11
A,1988-04-27,12
A,1990-08-31,12
A,1997-12-15,13
A,1994-10-25,13
A,1993-01-11,14
A,1992-08-22,14
A,1990-09-26,14
A,1993-05-08,16


In [15]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [16]:
!cat output/000000_0

A,1990-10-06,10
A,1990-09-05,11
A,1988-04-27,12
A,1990-08-31,12
A,1997-12-15,13
A,1994-10-25,13
A,1993-01-11,14
A,1992-08-22,14
A,1990-09-26,14
A,1993-05-08,16
A,1990-07-22,18
A,1992-09-19,18
B,1995-08-23,10
B,1999-06-11,12
B,1998-11-22,13
B,1999-10-21,13
B,1997-04-09,14
B,1995-09-06,14
B,1993-03-02,14
B,1999-08-28,14
B,1991-10-01,15
B,1994-08-30,17
C,1994-01-25,6
C,1994-07-27,7
C,1991-02-12,13
C,1994-09-09,15
D,1990-10-10,15
E,1994-02-14,5
E,1999-09-10,11
E,1999-12-06,12
E,1993-01-27,13
E,1991-02-18,14
E,1999-01-14,15
E,1990-05-03,16
E,1998-09-14,16
E,1985-02-12,16
E,1995-04-25,16
E,1993-07-21,17
E,1990-02-09,18
E,1993-12-27,18
